# Exploratory Notebook Examining the Candle and Candles Classes of aiomql

In [1]:
from pandas import Series, DataFrame
from aiomql import MetaTrader, TimeFrame, Candle, Candles, Config, Account

In [2]:
# a config file is in the notebooks folder
# if you don't wish to bother with a config file, simply call the login method below with your account details.
mt5 = MetaTrader()
init = await mt5.initialize()
login = await mt5.login()
if init and login:
    print("Terminal successfully initialized and login successful")

() {'login': 40311691, 'password': 'Aiomql-Demo1', 'server': 'Deriv-Demo', 'timeout': 60000, 'portable': False}
{'login': 40311691, 'password': 'Aiomql-Demo1', 'server': 'Deriv-Demo'}
40311691 Aiomql-Demo1 Deriv-Demo 60000
Terminal sucessfully initialized and login successful


## Candles Class

In [3]:
# get price bars from the terminal for the last 200 hours
# bars is a numpy.ndarray object
bars = await mt5.copy_rates_from_pos("EURUSD", TimeFrame.H1, 0, 200)

In [4]:
# it is an iterable and there for can passed in to the Candles class
candles = Candles(data=bars)

In [5]:
# the candles class creates a dataframe object from the price bars and saves it in an internal _data attribute which is
# accessible via the data property
print(isinstance(candles._data, DataFrame))
print(candles.data.head(5))


True
         time     open     high      low    close  tick_volume  spread  \
0  1731531600  1.05621  1.05668  1.05618  1.05632         1849       0   
1  1731535200  1.05578  1.05628  1.05570  1.05621         1188       8   
2  1731538800  1.05619  1.05658  1.05603  1.05657         1456       1   
3  1731542400  1.05657  1.05687  1.05626  1.05652         2382       1   
4  1731546000  1.05652  1.05652  1.05510  1.05560         3798       1   

   real_volume  
0            0  
1            0  
2            0  
3            0  
4            0  


In [6]:
# each of the column of the data attribute is in turn an attribute of the instance and returns a Series object
print(candles.open)

0      1.05621
1      1.05578
2      1.05619
3      1.05657
4      1.05652
        ...   
195    1.04439
196    1.04349
197    1.04540
198    1.04608
199    1.04779
Name: open, Length: 200, dtype: float64


In [7]:
# indexing a candles object will return a candle object
first_candle = candles[0]
last_candle = candles[-1]
print(first_candle) #  the index of the candle object will be 0 as it is the first in this sequence of candles
print(last_candle) #  the index here will be 199 as the 200th item

Candle(Index=0, time=1731531600.0, open=1.05621, high=1.05668, low=1.05618, close=1.05632)
Candle(Index=199, time=1732593600.0, open=1.04779, high=1.04874, low=1.04777, close=1.04854)


In [8]:
# slicing a candles object returns another candles object
first_100 = candles[:100]
print(len(first_100))
print(isinstance(first_100, Candles))
#  the index is reset
print(first_100.Index) 
print(first_100.data.index)

100
True
0      0
1      1
2      2
3      3
4      4
      ..
95    95
96    96
97    97
98    98
99    99
Length: 100, dtype: int64
RangeIndex(start=0, stop=100, step=1)


In [9]:
# attributes that are columns in the data property can be accessed using [] syntax
candles['close']

0      1.05632
1      1.05621
2      1.05657
3      1.05652
4      1.05560
        ...   
195    1.04349
196    1.04540
197    1.04608
198    1.04779
199    1.04854
Name: close, Length: 200, dtype: float64

In [10]:
# new columns and attributes can also be assigned using the [] syntax
candles['mid'] = (candles['close'] + candles['open']) / 2
# access mid as an attribute
candles.mid

0      1.056265
1      1.055995
2      1.056380
3      1.056545
4      1.056060
         ...   
195    1.043940
196    1.044445
197    1.045740
198    1.046935
199    1.048165
Name: mid, Length: 200, dtype: float64

In [11]:
# see columns of the data property with the colunmns property 
print(candles.columns)
print(candles.data.columns)

Index(['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread',
       'real_volume', 'mid'],
      dtype='object')
Index(['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread',
       'real_volume', 'mid'],
      dtype='object')


In [12]:
# iterating over the candles returns candle objects
count = 0
for candle in candles:
    print(isinstance(candle, Candle), candle)
    print(candle.Index == count, count)
    count += 1
    if count == 2:
        break


True Candle(Index=0, time=1731531600, open=1.05621, high=1.05668, low=1.05618, close=1.05632)
True 0
True Candle(Index=1, time=1731535200, open=1.05578, high=1.05628, low=1.0557, close=1.05621)
True 1


In [13]:
# get the timeframe of the candles 
print(candles.timeframe)

TIMEFRAME_H1


In [14]:
# the len of the candles 
len(candles)

200

In [15]:
# check contains property
candle_100 = candles[99]
candle_100 in candles

True

In [16]:
# technical analysis
# compute ema
# if append is true, the new result will be appended to the data property inplace
candles.ta.ema(length=20, append=True)

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
195    1.047810
196    1.047581
197    1.047438
198    1.047471
199    1.047573
Name: EMA_20, Length: 200, dtype: float64

In [17]:
candles.EMA_20

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
195    1.047810
196    1.047581
197    1.047438
198    1.047471
199    1.047573
Name: EMA_20, Length: 200, dtype: float64

In [18]:
# append is false, result will not be appended
candles.ta.sma(length=50)

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
195    1.046249
196    1.046219
197    1.046189
198    1.046202
199    1.046227
Name: SMA_50, Length: 200, dtype: float64

In [19]:
# this will fail
# candles.SMA_50

In [20]:
# some technical analysis operations cannot append to the dataframe, and needs the series to be passed in as arguments
# use the ta_lib properties for them
cae = candles.ta_lib.above(candles.close, candles.EMA_20)
print(cae)

0      0
1      0
2      0
3      0
4      0
      ..
195    0
196    0
197    0
198    1
199    1
Name: close_A_EMA_20, Length: 200, dtype: int32


In [21]:
# assign the result to the candles instance manually
candles['closeAema'] = cae
candles.columns

Index(['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread',
       'real_volume', 'mid', 'EMA_20', 'closeAema'],
      dtype='object')

In [22]:
# rename a column to a prefered name, this affects the attribute name also
candles.rename(EMA_20='ema')
candles.ema

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
195    1.047810
196    1.047581
197    1.047438
198    1.047471
199    1.047573
Name: ema, Length: 200, dtype: float64

In [23]:
# if inplace is false a new candles object is returned an the old one is untouched
n_candles = candles.rename(mid='avg_price', inplace=False)
print(isinstance(n_candles, Candles))
print(candles.columns, 'candles columns')
print(n_candles.columns, 'n_candles columns')

True
Index(['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread',
       'real_volume', 'mid', 'ema', 'closeAema'],
      dtype='object') candles columns
Index(['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread',
       'real_volume', 'avg_price', 'ema', 'closeAema'],
      dtype='object') n_candles columns


In [24]:
## Candle Class

In [25]:
# create a candle from a single price bar
bar = bars[0]
bar = {key: value for key, value in zip(('time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume'), bar)}
candle = Candle(**bar)

In [26]:
print(candle)

Candle(Index=0, time=1731531600, open=1.05621, high=1.05668, low=1.05618, close=1.05632)


In [27]:
# a candle can be iterated ove
for prop in candle:
    print(prop)

('time', 1731531600)
('Index', 0)
('real_volume', 0)
('spread', 0)
('tick_volume', 1849)
('open', 1.05621)
('high', 1.05668)
('low', 1.05618)
('close', 1.05632)


In [28]:
# turn a candle into a dict
candle.dict()

{'time': 1731531600,
 'Index': 0,
 'real_volume': 0,
 'spread': 0,
 'tick_volume': 1849,
 'open': 1.05621,
 'high': 1.05668,
 'low': 1.05618,
 'close': 1.05632}

In [29]:
# get only 'ohcl' properties
candle.dict(include={'open', 'high', 'close', 'low'})

{'open': 1.05621, 'high': 1.05668, 'low': 1.05618, 'close': 1.05632}

In [30]:
# values and keys method
values = candle.values()
print(values)
keys = candle.keys()
print(keys)

dict_values([1731531600, 0, 0, 0, 1849, 1.05621, 1.05668, 1.05618, 1.05632])
dict_keys(['time', 'Index', 'real_volume', 'spread', 'tick_volume', 'open', 'high', 'low', 'close'])


In [31]:
# __setitem__ and __getitem__ supported
candle['mid'] = (candle['high'] + candle.low) / 2
candle.mid

1.05643

In [32]:
# candles are compared by time
bar = {key: value for key, value in zip(('time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume'), bars[34])}
candle_2 = Candle(**bar)
candle_2 > candle

True

In [33]:
# check bullish an bearish
print(candle.is_bullish())
print(candle_2.is_bearish())

True
False


In [34]:
# set multiple attributes with set_attribute
candle.set_attributes(ema=1.3, sma=4.5)
candle.dict(include={'sma', 'ema'})

{'ema': 1.3, 'sma': 4.5}